In [ ]:
from htmd.ui import *

# Molecular dynamics protocols in HTMD

Summary: How to equilibrate and prepare a system for productive simulations

## Build a sample system

In [ ]:
tryp = Molecule("3PTB")
tryp.remove("resname BEN")
tryp_op = proteinPrepare(tryp)
tryp_seg = autoSegment(tryp_op)
tryp_solv = solvate(tryp_seg,pad=10)
#tryp_amber = amber.build(tryp_solv, param= ["frcmod.ionslrcm_cm_tip3p"], outdir='build-amber')
tryp_charmm = charmm.build(tryp_solv, outdir='build-charmm')

## Equilibration protocol

In [ ]:
from htmd.protocols.equilibration_v2 import Equilibration

In [ ]:
md = Equilibration()
md.runtime = 1000
md.timeunits = 'fs'
md.temperature = 300
md.useconstantratio = False  # only for membrane sims
# this is only needed for setting the flatbottom potential, otherwise remove it
# md.fb_reference = 'protein and resid 293'
# md.fb_selection = 'segname L and noh'
# md.fb_box = [-25, 25, -25, 25, 43, 45]
# md.fb_k = 5
md.write('./build-charmm/','./equil')

In [ ]:
%ls equil/

## Run the equilibration

In [ ]:
local = LocalGPUQueue()
local.submit('./equil/')
local.wait()

## Production protocol

In [ ]:
from htmd.protocols.production_v4 import Production

In [ ]:
md = Production()
md.runtime = 10
md.timeunits = 'ns'
md.temperature  = 300
md.acemd.bincoordinates = 'output.coor'
md.acemd.extendedsystem  = 'output.xsc'
md.write('equil','prod')

## Run the production

In [ ]:
local = LocalGPUQueue()
local.submit('./prod/')
local.wait()